In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from substrate_helpers import *

In [3]:
import sqlite3
import re
import datetime
import pandas as pd

In [4]:
from database_queries import update_db

In [5]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [6]:
con = sqlite3.connect('postthreadV1_write.db')
# con = sqlite3.connect('test1.db')
cur = con.cursor()

In [7]:
accounts = json.load(open("accounts.json", "r"))
schemas = json.load(open("schemas.json", "r"))

In [8]:
schemas

{'post': 1,
 'comment': 2,
 'vote': 3,
 'user': 4,
 'follow': 5,
 'link': 6,
 'payout': 7}

In [10]:
params = [
7,
{
"page_size": 10000,
"from_block": 0,
"to_block": 10000,
"from_index": 0,
}
]
request = substrate.rpc_request(
method='messages_getBySchema',
params=params,
)

SubstrateRequestException: {'code': 100, 'message': 'Dispatch Module Error', 'data': 'ModuleError { index: 61, error: [5, 0, 0, 0], message: None }'}

In [11]:
last_block_updated = update_db(start_block=0, backfill=True, schemaToUpdate=None)
con.commit()

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
        schemaValue = substrate.query(
            module='Schemas',
            storage_function='Schemas',
            params=[1]
        ).value

In [ ]:

date_format = "%Y-%m-%d %H:%M:%S"

In [ ]:
! cp postthreadV1_write.db postthreadV1_read.db

In [ ]:
con.close()

In [ ]:
    last_block = update_db(query_start=True,backfill=False, schemaToUpdate=None)

In [ ]:
    followers = "followers"
    user_msa_id = 2941
    
    if followers == "following":
        where_var = "protagonist_msa_id"
        select_var = "antagonist_msa_id"
    else:
        where_var = "antagonist_msa_id"
        select_var = "protagonist_msa_id"
        
    query = f'''
    SELECT followers.{followers}_msa_id, date_followed, user.*
    FROM user
    INNER JOIN (
        SELECT follow.{select_var} AS {followers}_msa_id, follow.date_minted AS date_followed
        FROM follow,
            (SELECT protagonist_msa_id, antagonist_msa_id, MAX(date_minted) AS date_minted
                FROM follow
                GROUP BY protagonist_msa_id, antagonist_msa_id) f2
        WHERE follow.protagonist_msa_id=f2.protagonist_msa_id 
        AND follow.antagonist_msa_id=f2.antagonist_msa_id
        AND follow.date_minted=f2.date_minted
        AND follow.event = 'follow'
        AND follow.{where_var} == {user_msa_id}
    ) followers
    ON user.msa_id_from_query = followers.{followers}_msa_id
    '''
    df = pd.read_sql_query(query, con)

In [ ]:
df

In [ ]:
a = []
a.append(1)

In [ ]:
a

In [ ]:
post_query = """
SELECT category, title, post.msa_id_from_query, username
FROM post 
JOIN user ON user.msa_id_from_query = post.msa_id_from_query
"""
all_posts = pd.read_sql_query(post_query, con)

In [ ]:

        all_users = pd.read_sql_query("SELECT msa_id_from_query, username, password FROM user", con)

In [ ]:
all_users[all_users['username'] == "Alice"].iloc[0].to_dict()

In [ ]:
    query = f'''
        SELECT *
        FROM user
    
    '''
    df = pd.read_sql_query(query, con)
    df

In [ ]:
    query = f'''
        SELECT DISTINCT(msa_id_from_query, ipfs_hash)
        FROM post
    '''
    df = pd.read_sql_query(query, con)
    df

In [ ]:

    query = f'''
        SELECT *
        FROM follow
        WHERE follow.antagonist_msa_id={8993}
    '''
    df = pd.read_sql_query(query, con)

In [ ]:
df

In [ ]:

    minutes_filter = 60*24
    date_where = ""
    if minutes_filter is not None:
        date_time = datetime.datetime.now() - datetime.timedelta(minutes=minutes_filter)
        date_str = date_time.strftime(date_format)
        date_where = f"post.date_minted >= '{date_str}'"

    order_by = ""
    if sort_by is None or sort_by == 'top':
        order_by = "ORDER BY total_votes DESC"
    if sort_by == 'new':
        order_by = "ORDER BY date_minted DESC"

    query = f'''
    SELECT * FROM post
    LEFT JOIN (
        SELECT parent_hash, SUM(num_votes) AS total_votes 
        FROM vote GROUP BY parent_hash
    ) votes ON votes.parent_hash = post.ipfs_hash
    LEFT JOIN (
        SELECT msa_id, username, password, profile_pic, wallet_ss58_address, block_number AS user_block_number, provider_key AS user_provider_key, date_minted AS user_date_minted
        FROM user u1
        WHERE date_minted = (SELECT max(date_minted) 
            FROM user u2
             WHERE u1.msa_id = u2.msa_id)
    ) users ON users.msa_id = post.msa_id_from_query
    WHERE {date_where}
    {order_by}
    '''
    df = pd.read_sql_query(query, con).drop_duplicates()
#     list(df.groupby('category')['total_votes'].count().sort_values(ascending=False).index)

In [ ]:
datetime.datetime.now()

In [ ]:
!npx crust-cli publish /tmp/posts/Qmd491pRqjeiaUZznQQKqCB8K4Sb9QX2WwzWkWqiLXSQBw

In [ ]:
df

In [ ]:
            query = f"""SELECT block_number FROM link ORDER BY date_minted DESC LIMIT 1"""
            start_block = int(pd.read_sql_query(query, con)['block_number'].iloc[0]) + 1

In [ ]:
start_block

In [ ]:
pd.read_sql_query("""SELECT * FROM link""", con)

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

In [ ]:

now = datetime.datetime.now()
tomorrow_time = (now + datetime.timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
(tomorrow_time - now).seconds

In [ ]:
datetime.datetime.now().strftime('%Y-%m-%d')

In [ ]:
(datetime.datetime.fromisoformat((datetime.datetime.now() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')) - datetime.datetime.now())

In [ ]:
date_time.seconds

In [ ]:
'2022-06-26 14:18:24' < date_time.strftime('%Y-%m-%d')

In [ ]:
def get_user_level_df(user_msa_id=None):
    if user_msa_id is None:
        where_msa = ""
    else:
        where_msa = f"WHERE msa_id_from_query = {user_msa_id}"
        
    query = f'''
    SELECT u1.*, posts.count AS post_count, comments.count AS comment_count, votes.count AS vote_count, 
            follows.count AS follow_count
    FROM user u1
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM post
        {where_msa}
        GROUP BY msa_id_from_query
    ) posts ON u1.msa_id_from_query = posts.msa_id_from_query 
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM comment
        {where_msa}
        GROUP BY msa_id_from_query
    ) comments ON u1.msa_id_from_query = comments.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM vote
        {where_msa}
        GROUP BY msa_id_from_query
    ) votes ON u1.msa_id_from_query = votes.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM follow
        {where_msa}
        GROUP BY msa_id_from_query
    ) follows ON u1.msa_id_from_query = follows.msa_id_from_query
    WHERE {"" if where_msa == "" else 'u1.' + where_msa.split('WHERE ')[1] + ' AND'} date_minted = (
        SELECT max(date_minted) 
        FROM user u2
        WHERE u1.msa_id = u2.msa_id
    )
    '''
    return df

In [ ]:
pd.read_sql_query("""SELECT block_number FROM post ORDER BY date_minted DESC LIMIT 1""", con)['block_number'].iloc[0]

In [ ]:
df = get_user_level_df(382)

In [ ]:
df.iloc[0].to_dict()

In [ ]:

accounts = {'Alice': 693, 'Charlie': 382, 'Dave': 694, 'Eve': 695, 'Ferdie': 696}

In [ ]:
post_hash

In [ ]:
    user_msa_id=None
    if user_msa_id is None:
        where_msa = ""
    else:
        where_msa = f"WHERE msa_id_from_query = {user_msa_id}"
        
    query = f'''
    SELECT u1.*, posts.count AS post_count, comments.count AS comment_count, votes.count AS vote_count, 
            follows.count AS follow_count
    FROM user u1
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM post
        {where_msa}
        GROUP BY msa_id_from_query
    ) posts ON u1.msa_id_from_query = posts.msa_id_from_query 
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM comment
        {where_msa}
        GROUP BY msa_id_from_query
    ) comments ON u1.msa_id_from_query = comments.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM vote
        {where_msa}
        GROUP BY msa_id_from_query
    ) votes ON u1.msa_id_from_query = votes.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM follow
        {where_msa}
        GROUP BY msa_id_from_query
    ) follows ON u1.msa_id_from_query = follows.msa_id_from_query
    WHERE {"" if where_msa == "" else 'u1.' + where_msa.split('WHERE ')[1] + ' AND'} date_minted = (
        SELECT max(date_minted) 
        FROM user u2
        WHERE u1.msa_id = u2.msa_id
    )
    '''
    df = pd.read_sql_query(query, con)
    df

In [ ]:
df.sort_values("post_count", ascending=False)

In [ ]:
import time
time.time()

In [ ]:
df[df['post_count'] > 0]

In [ ]:
query = """
    SELECT * FROM post 
    LEFT JOIN (
        SELECT parent_hash, SUM(num_votes) AS total_votes 
        FROM vote GROUP BY parent_hash
    ) votes ON votes.parent_hash = post.ipfs_hash
    LEFT JOIN (
        SELECT msa_id, username, password, profile_pic, wallet_ss58_address, block_number AS user_block_number, provider_key AS user_provider_key, date_minted AS user_date_minted
        FROM user u1
        WHERE date_minted = (SELECT max(date_minted) 
            FROM user u2
             WHERE u1.msa_id = u2.msa_id)
    ) users ON users.msa_id = post.msa_id_from_query
    WHERE post.ipfs_hash = 'QmbG2dxLWHKvdgsTqDzLjhD96BXrLQUeJkrLuasmPZKgn9'
    ORDER BY total_votes DESC
    """
df = pd.read_sql_query(query, con)
df

In [ ]:
query = '''
SELECT * 
FROM post
'''
df = pd.read_sql_query(query, con)
df

In [ ]:
query = '''
SELECT * 
FROM follow
WHERE protagonist_msa_id = 1335 
ORDER BY protagonist_msa_id
LIMIT 10
'''
df = pd.read_sql_query(query, con)
df

In [ ]:
user_msa_id = 1355
query = f'''
SELECT *
FROM post
'''
df = pd.read_sql_query(query, con)
df

In [ ]:
user_msa_id = 1355
query = f'''
SELECT user.*, posts.count AS post_count, comments.count AS comment_count, votes.count AS vote_count, 
        follows.count AS follow_count
FROM user
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM post
    WHERE msa_id_from_query = {user_msa_id}
) posts ON user.msa_id_from_query = posts.msa_id_from_query 
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM comment
    WHERE msa_id_from_query = {user_msa_id}
) comments ON user.msa_id_from_query = comments.msa_id_from_query
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM vote
    WHERE msa_id_from_query = {user_msa_id}
) votes ON user.msa_id_from_query = votes.msa_id_from_query
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM follow
    WHERE msa_id_from_query = {user_msa_id}
) follows ON user.msa_id_from_query = follows.msa_id_from_query
WHERE user.msa_id_from_query = {user_msa_id} 
'''
df = pd.read_sql_query(query, con).fillna(0)
df

In [ ]:
query = '''
SELECT * FROM vote
'''
df = pd.read_sql_query(query, con)
df

In [ ]:
query = '''
SELECT * FROM user
'''
df = pd.read_sql_query(query, con)
df

In [ ]:
import uvicorn
uvicorn.server.Config?

In [ ]:
query = '''
SELECT * FROM comment
'''
df = pd.read_sql_query(query, con)
df

In [ ]:
query = '''
SELECT DISTINCT(category) FROM post
'''
df = pd.read_sql_query(query, con)
df

In [ ]:
    page_number = 1
    num_posts = 10
    category = None
    sort_by = "top"
    minutes_filter = None
    user_msa_id = 1


    if minutes_filter is None:
        minutes_filter = 60*24*100

    category_where = ""
    if category:
        category_where = f"AND post.category = '{category}'"

    user_where = ""
    if user_msa_id:
        user_where = f"AND post.msa_id_from_query = {user_msa_id}"

    order_by = ""
    if sort_by is None or sort_by == 'top':
        order_by = "ORDER BY total_votes DESC"
    if sort_by == 'new':
        order_by = "ORDER BY date_minted DESC"

    lower_bound = (page_number - 1) * num_posts
    upper_bound = page_number * num_posts

    date_format = "%m-%d-%Y %H:%M:%S"
#     date_format = "%Y-%m-%d %H:%M:%S"
    date_time = datetime.datetime.now() - datetime.timedelta(minutes=minutes_filter)
    date_str = date_time.strftime(date_format)

    query = f'''
    SELECT * FROM post 
    JOIN (
        SELECT parent_hash, SUM(num_votes) AS total_votes 
        FROM vote GROUP BY parent_hash
    ) votes ON votes.parent_hash = post.ipfs_hash
    WHERE post.date_minted >= '{date_str}' {category_where} {user_where}
    {order_by}
    LIMIT {lower_bound}, {upper_bound}
    '''
    df = pd.read_sql_query(query, con)
    df

In [ ]:
print(query)

In [ ]:
for row in cur.execute("PRAGMA table_info('post')"):
    print(row)

In [ ]:
for schema, contents in content_jsons.items():
    schema_items = schema.split(',')
    
    for content in contents:
        ipfs_hash = bytes.fromhex(content['data'][2:]).decode()
            
        data = json.loads(client.cat(ipfs_hash).decode())

In [ ]:
def subscription_handler(obj, update_nr, subscription_id):
    last_block_updated = update_db(start_block=obj['header']['number'], backfill=False)
    con.commit()
    print(last_block_updated)


result = substrate.subscribe_block_headers(subscription_handler, include_author=False)

In [ ]:
def subscription_handler(obj, update_nr, subscription_id):
    return obj, update_nr, subscription_id
result = substrate.subscribe_block_headers(subscription_handler, include_author=False)

In [ ]:
e = substrate.get_events(substrate.get_block_hash(result[0]['header']['number']))

In [ ]:
substrate.

In [ ]:
e[1].value

In [ ]:
result[0]

In [ ]:
result[0]['header']['digest']['logs'][0]

In [ ]:
date_str = None
for extrinsic in substrate.get_block(substrate.get_block_hash(2))['extrinsics']:
    if "Timestamp" == extrinsic.value['call']['call_module']:
        timestamp = extrinsic.value['call']['call_args'][0]['value']
        date_time = datetime.fromtimestamp(timestamp/1000)
        date_str = date_time.strftime("%m/%d/%Y, %H:%M:%S")
        break
if date_str is None:
    print('Fail')
date_str

In [ ]:
extrinsic = substrate.get_block(substrate.get_block_hash(2))['extrinsics'][1]

In [ ]:
extrinsic.value['call']